<a href="https://colab.research.google.com/github/profesorKiki/EjemplosPython/blob/main/EjemplosSincronismo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

C++

Prueba con variable no atómica

In [5]:
%%writefile suma.cpp
#include <iostream>
#include <thread>
#include <vector>

int count=0;

void Sum()
{
    for (int i=0; i<100; ++i)
    {
        count++;
    }
}

int main()
{
    std::vector<std::thread> threads;

    for(int i=0; i<1000; ++i)
    {
        threads.push_back( std::thread(Sum) );
    }

    for(int i=0; i<1000; ++i)
    {
        threads[i].join();
    }

    std::cout<<"Resultado "<<count<<std::endl;

    return EXIT_SUCCESS;
}

Writing suma.cpp


In [6]:
!g++ -o suma suma.cpp

In [9]:
!./suma

Resultado 99743


Prueba con variable atómica

In [8]:
%%writefile suma_atomica.cpp
#include <iostream>
#include <thread>
#include <vector>
#include <atomic>

std::atomic<int> count(0);

void Sum()
{
    for (int i=0; i<100; ++i)
    {
        atomic_fetch_add(&count, 1);
    }
}

int main()
{
    std::vector<std::thread> threads;

    for(int i=0; i<1000; ++i)
    {
        threads.push_back( std::thread(Sum) );
    }

    for(int i=0; i<1000; ++i)
    {
        threads[i].join();
    }

    std::cout<<"Resultado "<<count.load()<<std::endl;

    return EXIT_SUCCESS;
}

Writing suma_atomica.cpp


In [9]:
!g++ -o suma_atomica suma_atomica.cpp

In [16]:
!./suma_atomica

Resultado 100000


Python garantiza atomicidad en las operaciones sobre los tipos de datos enteros y flotantes, no así para los siguientes tipos:<br>

listas -> [2, 4, 8]<br>
diccionarios -> {'a': 2, 'b': 4}<br>
conjuntos -> {2, 4, 8}<br>
cadenas y objetos personalizados.<br>
<br>
Con lo cual es recomendable garantizar la mutua exclusión con algún mecanismo de sincronización.

Python - Lock + with

In [4]:
import threading

MAX_THREADS = 1000
lock = threading.Lock()
items = []

def sum():
    global items
    for _ in range(100):
        with lock:
          items.append(1)

def main():
    my_threads = []

    for _ in range(MAX_THREADS):
        my_threads.append(threading.Thread(target=sum))

    for i in range(MAX_THREADS):
        my_threads[i].start()

    for i in range(MAX_THREADS):
        my_threads[i].join()

    print( "Cantidad de elementos: ", len(items) )

if __name__ == '__main__':
    main()



Cantidad de elementos:  100000


Java: Wait y Notify

In [16]:
%%writefile WaitNotify.java
public class WaitNotify
{
    private static final int THREADS = 10;
    private static final Object lock = new Object();
    private static boolean condition = false;
    private static int turn = 0;

    static class MyThread extends Thread
    {
        private int id;

        public MyThread(int id)
        {
            this.id = id;
        }

        public void run()
        {
            synchronized(lock)
            {
                while(turn != id)
                {
                    try
                    {
                        lock.wait();
                    }
                    catch(InterruptedException e)
                    {
                        e.printStackTrace();
                    }
                }
                System.out.println("Hilo " + this.id);
                turn++;
                lock.notifyAll();
            }
        }
    }

    public static void main(String[] args) throws InterruptedException
    {
        MyThread []threads = new MyThread[THREADS];

        for(int i=0; i<THREADS; ++i)
        {
            threads[i] = new MyThread(i);
        }

        for(int i=0; i<THREADS; ++i)
        {
            threads[i].start();
        }

        for(int i=0; i<THREADS; ++i)
        {
            threads[i].join();
        }

        System.out.println("Fin del main ");
    }
}

Overwriting WaitNotify.java


In [18]:
!java WaitNotify.java

Hilo 0
Hilo 1
Hilo 2
Hilo 3
Hilo 4
Hilo 5
Hilo 6
Hilo 7
Hilo 8
Hilo 9
Fin del main 


Python: Events

In [37]:
import threading
#import time
MAX_THREADS = 10
event = threading.Event()
turn = 0

def thread_in_order(id):
    global turn

    while turn != id:
      event.wait()
    #time.sleep(1)
    print( f"Hilo: {id}" )
    turn+=1
    event.set()


def main():
    my_threads = []

    for i in range(MAX_THREADS):
        my_threads.append(threading.Thread(target=thread_in_order,args=(i,)))

    for i in range(MAX_THREADS):
        my_threads[i].start()

    for i in range(MAX_THREADS):
        my_threads[i].join()

    print( "Fin del main" )

if __name__ == '__main__':
    main()

Hilo: 0
Hilo: 1
Hilo: 2
Hilo: 3
Hilo: 4
Hilo: 5
Hilo: 6
Hilo: 7
Hilo: 8
Hilo: 9
Fin del main
